In [1]:
#!pip install qiskit
#!pip install qiskit_braket_provider
#!pip install tadasets
#!pip install gudhi

In [2]:
from qiskit_braket_provider import BraketLocalBackend, AWSBraketProvider
from qiskit.extensions import HamiltonianGate
from qiskit import QuantumCircuit, transpile
import numpy as np
import tadasets
from classical_betti_calc import boundary, homology, betti
from utils import make_simplicies,gershgorin
import matplotlib.pyplot as plt

np.random.seed(0)

In [3]:
print("start")
provider = AWSBraketProvider()
local_simulator = BraketLocalBackend()
aws_simulator_sv = provider.get_backend("SV1")
aws_simulator_dm = provider.get_backend("dm1")

# Select simulator to use
simulator_to_use = aws_simulator_sv

start


In [4]:
shots = 10000

In [5]:
def qft_dagger(qc, n):
    """n-qubit QFTdagger the first n qubits in circ"""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi / float(2 ** (j - m)), m, j)
        qc.h(j)

In [6]:
# For use with a simulator, we consider demonstrative examples with few points
# 20 points are sampled for both shapes and embedded to a 10D ambient space
example_torus = tadasets.torus(n=20, c=5, a=1, ambient=10, noise=0)
example_swiss_roll = tadasets.swiss_roll(n=20, r=5, ambient=10, noise=0)

scs = make_simplicies([example_torus, example_swiss_roll], 2, 5)

In [7]:
for j, sc in enumerate(scs):
    bnd, simplicies = boundary(sc)
    b = betti(homology(bnd))

    if j < 1:
        print("The Betti number 0 of the Torus is", b[0])
    else:
        print("The Betti number 0 of the Swiss roll is", b[0])

    for precision_qubits in range(1, 4):
        k = 0  # We are only estimating Betti number 0
        comb_lap = bnd[k].T @ bnd[k] + bnd[k + 1] @ bnd[k + 1].T
        num = int(np.ceil(np.log2(comb_lap.shape[0])))
        max_eig = max(gershgorin(comb_lap), 1)
        H = np.identity(2**num) * max_eig / 2
        H[: comb_lap.shape[0], : comb_lap.shape[0]] = comb_lap
        H = H / max_eig * 6
        U = HamiltonianGate(H, -1)
        qc = QuantumCircuit(2 * num + precision_qubits, precision_qubits)
        qc.h(range(precision_qubits, num + precision_qubits))
        for i in range(num):
            qc.cx(i + precision_qubits, i + num + precision_qubits)
        qc.barrier()
        qc.h(range(precision_qubits))
        for i in range(1, precision_qubits + 1):
            qc.append(
                (U.power(2 ** (precision_qubits - i))).control(1),
                [i - 1] + list(range(precision_qubits, num + precision_qubits)),
            )
        qc.barrier()
        qft_dagger(qc, precision_qubits)
        qc.barrier()
        qc.measure(range(precision_qubits), range(precision_qubits))

        qc2 = transpile(qc, simulator_to_use, seed_transpiler=0)

        task = simulator_to_use.run(qc2, shots=shots)
        counts = task.result().get_counts()
        zero_counts = 0
        for state, count in counts.items():
            if state[::-1][:precision_qubits] == "0" * precision_qubits:
                zero_counts = zero_counts + count
        qb_k = (zero_counts / shots) * 2**num
        
        if j < 1:
            print(
                "The estimated Betti number (k=0) of the Torus is",
                qb_k,
                "using",
                precision_qubits,
                "qubits for estimation in QPE",
            )
        else:
            print(
                "The estimated Betti number (k=0) of the Swiss roll is",
                qb_k,
                "using",
                precision_qubits,
                "qubits for estimation in QPE",
            )

The Betti number 0 of the Torus is 7
The estimated Betti number (k=0) of the Torus is 11.8592 using 1 qubits for estimation in QPE
The estimated Betti number (k=0) of the Torus is 9.2672 using 2 qubits for estimation in QPE
The estimated Betti number (k=0) of the Torus is 7.4816 using 3 qubits for estimation in QPE
The Betti number 0 of the Swiss roll is 16
The estimated Betti number (k=0) of the Swiss roll is 20.1472 using 1 qubits for estimation in QPE
The estimated Betti number (k=0) of the Swiss roll is 19.504 using 2 qubits for estimation in QPE
The estimated Betti number (k=0) of the Swiss roll is 18.8928 using 3 qubits for estimation in QPE


In [8]:
print("Finish")

Finish


When using the local_simulator we got these results:
```
start
The Betti number 0 of the Torus is 7
The estimated Betti number (k=0) of the Torus is 12.144 using 1 qubits for estimation in QPE
The estimated Betti number (k=0) of the Torus is 9.6512 using 2 qubits for estimation in QPE
The estimated Betti number (k=0) of the Torus is 7.7344 using 3 qubits for estimation in QPE
The estimated Betti number (k=0) of the Torus is 7.1648 using 4 qubits for estimation in QPE
The estimated Betti number (k=0) of the Torus is 7.1456 using 5 qubits for estimation in QPE
The estimated Betti number (k=0) of the Torus is 7.1264 using 6 qubits for estimation in QPE
The estimated Betti number (k=0) of the Torus is 6.976 using 7 qubits for estimation in QPE
The estimated Betti number (k=0) of the Torus is 7.072 using 8 qubits for estimation in QPE
The Betti number 0 of the Swiss roll is 16
The estimated Betti number (k=0) of the Swiss roll is 19.84 using 1 qubits for estimation in QPE
The estimated Betti number (k=0) of the Swiss roll is 19.3408 using 2 qubits for estimation in QPE
The estimated Betti number (k=0) of the Swiss roll is 18.5536 using 3 qubits for estimation in QPE
The estimated Betti number (k=0) of the Swiss roll is 16.3904 using 4 qubits for estimation in QPE
The estimated Betti number (k=0) of the Swiss roll is 16.2368 using 5 qubits for estimation in QPE
The estimated Betti number (k=0) of the Swiss roll is 16.0256 using 6 qubits for estimation in QPE
The estimated Betti number (k=0) of the Swiss roll is 15.6896 using 7 qubits for estimation in QPE
The estimated Betti number (k=0) of the Swiss roll is 15.9104 using 8 qubits for estimation in QPE
Finish
```


<img src='images/torus_res.png' />
<br>
<img src='images/swiss_res.png' />

This demonstrates that the answer converges to the correct value with increasing number of precision qubits.